In [0]:
%pip install 'databricks-sdk==0.61.0' 'pyarrow<20' 'databricks-sdk[notebook]' 'databricks-agents==1.2.0' 'mlflow<=3.1' 'mlflow[databricks]' 'databricks-vectorsearch==0.57' 'langchain==0.3.27' 'langchain-mcp' 'langchain_core==0.3.74' 'databricks-langchain==0.7.0' 'bs4' 'dotenv' 'psycopg2-binary==2.9.7' 'pgvector==0.2.5' 'langgraph==0.3.4'
import os
if os.environ.get("DATABRICKS_RUNTIME_VERSION"):
    dbutils.library.restartPython()

INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Downl

*** WARNING: max output size exceeded, skipping output. ***

y<4.0.0,>=3.1.0->databricks-agents==1.2.0) (3.11.0)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.6/680.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [0]:
dbutils.widgets.text("embedding_model", "databricks-gte-large-en")
dbutils.widgets.text("database_instance_name", "tannerw-adtech-db")
dbutils.widgets.text("postgres_database_name", "databricks_postgres")
dbutils.widgets.text("llm_model_serving_endpoint_name", "databricks-claude-3-7-sonnet")
dbutils.widgets.text("target_catalog", "tanner_wendland")
dbutils.widgets.text("target_schema", "default")
dbutils.widgets.text("genie_space_id", "01efcca6fdc712d7be87a40ad4a2e33e")

In [0]:
embedding_model = dbutils.widgets.get("embedding_model")
database_instance_name = dbutils.widgets.get("database_instance_name")
postgres_database_name = dbutils.widgets.get("postgres_database_name")
llm_model_serving_endpoint_name = dbutils.widgets.get("llm_model_serving_endpoint_name")
target_catalog = dbutils.widgets.get("target_catalog")
target_schema = dbutils.widgets.get("target_schema")
genie_space_id = dbutils.widgets.get("genie_space_id")

In [0]:
import functools
import os
from typing import Any, Generator, Literal, Optional

In [0]:
from databricks.sdk import WorkspaceClient
import os
from typing import Optional
import uuid

host = os.environ.get("DATABRICKS_HOST")
token = os.environ.get("DATABRICKS_TOKEN")

workspace_client = WorkspaceClient(host=host, token=token)

def get_postgres_connection(
    client: WorkspaceClient,
    db_name: str,
    database_name: Optional[str] = "databricks_postgres"
) -> str:
    """
    Get PostgreSQL connection string using Databricks SDK.

    Args:
        client (WorkspaceClient): The Databricks workspace client.
        db_name (str): The name of the database instance.
        database_name (Optional[str], optional): The name of the database to connect to.
            Defaults to "databricks_postgres".

    Returns:
        str: PostgreSQL connection string.
    """
    database = client.database.get_database_instance(db_name)
    credentials = client.database.generate_database_credential(
        instance_names=[db_name],
        request_id=str(uuid.uuid4())
    )

    # Use POSTGRES_GROUP env var as username if set, otherwise use current user
    postgres_group = os.getenv('POSTGRES_GROUP')
    username = postgres_group if postgres_group else client.current_user.me().user_name

    database_info = {
        "host": database.read_write_dns,
        "port": "5432",
        "database": database_name,
        "username": username,
        "password": credentials.token,
        "ssl_mode": "require"
    }

    database_url = (
        f"postgresql://{database_info['username']}:{database_info['password']}"
        f"@{database_info['host']}:{database_info['port']}/"
        f"{database_info['database']}?sslmode={database_info['ssl_mode']}"
    )

    return database_url

database_url = get_postgres_connection(workspace_client, database_instance_name, postgres_database_name)

In [0]:
import mlflow
mlflow.set_registry_uri("databricks-uc")
mlflow.autolog()
mlflow.langchain.autolog()

## Chain Config

In [0]:
chain_config = {
    "llm_model_serving_endpoint_name": llm_model_serving_endpoint_name,
    "embedding_model": embedding_model,
    "database_instance_name": database_instance_name,
    "postgres_database_name": postgres_database_name,
    "genie_space_id": genie_space_id,
    "genie_agent_description": "You are a an agent that can invoke Genie, a powerful text-to-sql database assistant. You can use this tool to answer questions related to sales pipelines.",
    "general_assistant_description": "The General Assistant agent is a helpful assistant that can answer any question.",
    "code_agent_description": "The Coder agent specializes in solving programming challenges, generating code snippets, debugging issues, and explaining complex coding concepts.",
    "llm_prompt_template": """You are an assistant that answers questions. You have access to a vector search tool that searches previous conversations, but you should primarily use the immediate conversation context provided. 

Current conversation context: {context}

Instructions:
1. First, use the current conversation context above to answer questions
2. Only use the vector search tool if the question explicitly refers to information from previous conversations that is NOT in the current context
3. If the current conversation context contains sufficient information to answer the question, do NOT use the vector search tool
4. The vector search tool should be used sparingly, only when the user is clearly asking about something from their chat history that isn't in the current conversation""",
}

## Vector Config

In [0]:
retriever_config = {
    "parameters": {
        "k": 3
    }
}

In [0]:
from typing import Dict

# combine dynamic and static filters for vector search
def create_configurable_with_filters(input: Dict, retriever_config: Dict) -> Dict:
   """
   create configurable object with filters.
   Args:
       input: The input data containing filters.
   Returns:
       A configurable object with filters added to the search_kwargs.
   """
   if "custom_inputs" in input:
       filters = input["custom_inputs"]["filters"]
   else:
       filters = {}
   print(filters)
   configurable = {
       "configurable": {
           "search_kwargs": {
               "k": retriever_config.get("parameters")["k"],
               "filter": filters
           }
       }
   }
   return configurable

## Simple Chains

In [0]:
from typing import Optional, Dict, Any, List
from databricks_langchain import DatabricksEmbeddings
from sqlalchemy import create_engine, text, event
from pgvector.psycopg2 import register_vector

# Use psycopg2 driver explicitly for SQLAlchemy
database_url_sqlalchemy = database_url.replace("postgresql://", "postgresql+psycopg2://")

engine = create_engine(database_url_sqlalchemy, pool_pre_ping=True)

@event.listens_for(engine, "connect")
def _register_vector(dbapi_connection, connection_record):
    # Register pgvector adapter for psycopg2 so Python lists map to the vector type
    register_vector(dbapi_connection)

def pg_vector_similarity_search(
    embeddings: DatabricksEmbeddings,
    query: str,
    k: int = 3,
    filters: Optional[Dict[str, Any]] = None,
) -> str:
    # 1) Embed the query
    query_embedding = embeddings.embed_query(query)

    # 2) Build where clause from filters
    filters = filters or {}
    where_conditions: List[str] = []
    params: Dict[str, Any] = {}

    if "user_name" in filters:
        where_conditions.append("me.user_name = :user_name")
        params["user_name"] = filters["user_name"]

    if "chat_id" in filters:
        where_conditions.append("me.chat_id = :chat_id")
        params["chat_id"] = filters["chat_id"]

    where_clause = ""
    if where_conditions:
        where_clause = "WHERE " + " AND ".join(where_conditions)

    # 3) Query using cosine distance operator (<=>); requires pgvector
    sql = text(f"""
        SELECT
            ch.message_content,
            me.user_name,
            me.chat_id,
            ch.message_type,
            ch.created_at,
            ch.message_order,
            (me.embedding <=> CAST(:query_embedding AS vector)) AS distance
        FROM message_embeddings me
        JOIN chat_history ch ON me.message_id = ch.id
        {where_clause}
        ORDER BY me.embedding <=> CAST(:query_embedding AS vector)
        LIMIT :k
    """)

    # 4) Execute
    with engine.connect() as conn:
        rows = conn.execute(
            sql, {"query_embedding": query_embedding, "k": k, **params}
        ).fetchall()

    # 5) Format context for the prompt
    passages = [f"Passage: {r.message_content}" for r in rows]
    return "\n".join(passages)

In [0]:
from langchain.tools import Tool
from langchain_core.prompts import ChatPromptTemplate
from databricks_langchain.chat_models import ChatDatabricks
from langchain.tools import Tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from operator import itemgetter
from langchain.schema.runnable import RunnableLambda
from databricks_langchain.genie import GenieAgent
from langchain_core.runnables import RunnableLambda
from langgraph.graph import END, StateGraph
from langgraph.graph.state import CompiledStateGraph
from langgraph.prebuilt import create_react_agent
from mlflow.langchain.chat_agent_langgraph import ChatAgentState
from databricks_langchain import (
    ChatDatabricks,
    UCFunctionToolkit,
    DatabricksFunctionClient,
    set_uc_function_client
)
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import (
    ChatAgentChunk,
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)
from pydantic import BaseModel

## Load the chain's configuration
model_config = mlflow.models.ModelConfig(development_config=chain_config)

def create_context_aware_vector_search_tool(state):
    """Create a vector search tool that has access to user context from state"""
    
    def filtered_vector_search(query: str) -> str:
        # Extract user context from state
        user_context = state.get("user_context", {})
        filters = user_context.get("filters", {})
        
        # Use your existing pg_vector_similarity_search with filters
        return pg_vector_similarity_search(
            embeddings=embeddings, 
            query=query, 
            k=retriever_config["parameters"]["k"], 
            filters=filters
        )
    
    return Tool(
        name="search_chat_history",
        description="Retrieve chat history from Postgres (pgvector) for the current user; use only if the immediate conversation context is insufficient. THe input to this function should be the user message.",
        func=filtered_vector_search,
    )


genie_agent_description = model_config.get('genie_agent_description')
general_assistant_description = model_config.get('general_assistant_description')
code_agent_description = model_config.get('code_agent_description')

genie_agent = GenieAgent(
    genie_space_id=model_config.get('genie_space_id'),
    genie_agent_name="Genie",
    description=genie_agent_description,
    client=workspace_client,
    include_context=True,
)

# Max number of interactions between agents
MAX_ITERATIONS = 3

worker_descriptions = {
    "Genie": genie_agent_description,
    "General": general_assistant_description,
    "Coder": code_agent_description,
}

formatted_descriptions = "\n".join(
    f"- {name}: {desc}" for name, desc in worker_descriptions.items()
)

system_prompt = f"Decide between routing between the following workers or ending the conversation if an answer is provided. \n{formatted_descriptions}"
options = ["FINISH"] + list(worker_descriptions.keys())
FINISH = {"next_node": "FINISH"}

# Updated prompt template for tool-calling agent -- TODO REMOVE
# prompt = ChatPromptTemplate.from_messages([
#     ("system", model_config.get('llm_prompt_template')),
#     ("user", "{question}"),
#     ("placeholder", "{agent_scratchpad}"),
# ])

# Our foundation model answering the final prompt
model = ChatDatabricks(
    endpoint=model_config.get("llm_model_serving_endpoint_name"),
    extra_params={"temperature": 0.01, "max_tokens": 500}
)

tools = []

# TODO if desired, add additional tools and update the description of this agent
uc_tool_names = ["system.ai.*"]
uc_toolkit = UCFunctionToolkit(function_names=uc_tool_names)
tools.extend(uc_toolkit.tools)

embeddings = DatabricksEmbeddings(
    endpoint=chain_config["embedding_model"],
    token=os.environ.get("DATABRICKS_TOKEN")
)

def supervisor_agent(state):
    count = state.get("iteration_count", 0) + 1
    if count > MAX_ITERATIONS:
        return FINISH
    
    class nextNode(BaseModel):
        next_node: Literal[tuple(options)]

    preprocessor = RunnableLambda(
        lambda state: [{"role": "system", "content": system_prompt}] + state["messages"]
    )
    supervisor_chain = preprocessor | model.with_structured_output(nextNode)
    next_node = supervisor_chain.invoke(state).next_node
    
    # if routed back to the same node, exit the loop
    if state.get("next_node") == next_node:
        return FINISH
    return {
        "iteration_count": count,
        "next_node": next_node
    }

#######################################
# Define our multiagent graph structure
#######################################


def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {
        "messages": [
            {
                "role": "assistant",
                "content": result["messages"][-1].content,
                "name": name,
            }
        ]
    }


def final_answer(state):
    prompt = "Using only the content in the messages, respond to the previous user question using the answer given by the other assistant messages."
    preprocessor = RunnableLambda(
        lambda state: state["messages"] + [{"role": "user", "content": prompt}]
    )
    final_answer_chain = preprocessor | model
    return {"messages": [final_answer_chain.invoke(state)]}


def agent_node_with_context(state, agent, name):
    """Enhanced agent node that injects context-aware tools"""
    
    # Create the shared vector search tool with current state context
    vector_search_tool = create_context_aware_vector_search_tool(state)
    
    if name == "Genie":
        # Genie already has its tools, just add vector search
        enhanced_agent = agent  # Genie agent already configured
        # Note: GenieAgent might need special handling - see option below
        
    elif name == "Coder" or name == "General":
        # Add vector search tool to Coder's existing UC tools
        enhanced_tools = tools + [vector_search_tool]  # tools is your UC toolkit
        enhanced_agent = create_react_agent(model, tools=[vector_search_tool])
        
    # Execute with enhanced agent
    result = enhanced_agent.invoke(state)
    return {
        "messages": [{
            "role": "assistant",
            "content": result["messages"][-1].content,
            "name": name,
        }]
    }

# Create enhanced agent nodes
def enhanced_genie_node(state):
    enhanced_agent = genie_agent
    return agent_node_with_context(state, enhanced_agent, "Genie")

def enhanced_coder_node(state):
    return agent_node_with_context(state, None, "Coder")

def enhanced_general_node(state):
    return agent_node_with_context(state, None, "General")

class AgentState(ChatAgentState):
    next_node: str
    iteration_count: int
    user_context: Optional[Dict[str, Any]] = None

workflow = StateGraph(AgentState)
# Agent States
workflow.add_node("Genie", enhanced_genie_node)
workflow.add_node("Coder", enhanced_coder_node)
workflow.add_node("General", enhanced_general_node)
# Supervisor States
workflow.add_node("supervisor", supervisor_agent)
workflow.add_node("final_answer", final_answer)

workflow.set_entry_point("supervisor")
# We want our workers to ALWAYS "report back" to the supervisor when done
for worker in worker_descriptions.keys():
    workflow.add_edge(worker, "supervisor")

# Let the supervisor decide which next node to go
workflow.add_conditional_edges(
    "supervisor",
    lambda x: x["next_node"],
    {**{k: k for k in worker_descriptions.keys()}, "FINISH": "final_answer"},
)
workflow.add_edge("final_answer", END)
multi_agent = workflow.compile()

###################################
# Wrap our multi-agent in ChatAgent
###################################


class LangGraphChatAgent(ChatAgent):
    def __init__(self, agent: CompiledStateGraph):
        self.agent = agent

    def predict(
    self,
    messages: list[ChatAgentMessage],
    context: Optional[ChatContext] = None,
    custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        # Extract user context from custom_inputs
        user_context = {}
        if custom_inputs and "filters" in custom_inputs:
            user_context["filters"] = custom_inputs["filters"]
        
        request = {
            "messages": [m.model_dump_compat(exclude_none=True) for m in messages],
            "user_context": user_context  # Inject user context into state
        }

        messages = []
        for event in self.agent.stream(request, stream_mode="updates"):
            for node_data in event.values():
                messages.extend(
                    ChatAgentMessage(**msg) for msg in node_data.get("messages", [])
                )
        return ChatAgentResponse(messages=messages)

    def predict_stream(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> Generator[ChatAgentChunk, None, None]:
        request = {
            "messages": [m.model_dump_compat(exclude_none=True) for m in messages]
        }
        for event in self.agent.stream(request, stream_mode="updates"):
            for node_data in event.values():
                yield from (
                    ChatAgentChunk(**{"delta": msg})
                    for msg in node_data.get("messages", [])
                )


# Create the agent object, and specify it as the agent object to use when
# loading the agent back for inference via mlflow.models.set_model()
AGENT = LangGraphChatAgent(multi_agent)
# mlflow.models.set_model(AGENT)

# def extract_user_query_string(chat_messages_array):
#     return chat_messages_array[-1]["content"]

# def extract_context_string(chat_messages_array):
#     return '\n'.join([f"Role: {message['role']} - Content: {message['content']}" for message in chat_messages_array[:-1]])

# # In your rag_chain_with_tool():
# def rag_chain_with_tool(input_data: dict) -> str:
#     user_query = extract_user_query_string(input_data["messages"])
#     def filtered_vector_search(q: str) -> str:
#         return pg_vector_search_with_filters(embeddings, q, input_data)

#     filtered_tool = Tool(
#         name="search_chat_history",
#         description="Retreive chat history from this vector database for the current user, use this tool to answer questions that may refer to previous conversations",
#         func=filtered_vector_search,
#     )

#     filtered_agent = create_tool_calling_agent(model, [filtered_tool], prompt)
#     filtered_agent_executor = AgentExecutor(agent=filtered_agent, tools=[filtered_tool], verbose=True)
#     context = extract_context_string(input_data["messages"])
#     result = filtered_agent_executor.invoke({"question": user_query, "context": context})
#     return result["output"]

# # Create a runnable version of the chain
# chain = RunnableLambda(rag_chain_with_tool)

Trace(trace_id=tr-25625ff6fe1c9e700295633d20499706)

## Test Document Retreival

In [0]:
input_example_no_filter = {"messages": [ {"role": "user", "content": "Have we talked about databricks before?"}]}
answer_no_filter = AGENT.predict(input_example_no_filter)
print(answer_no_filter)

messages=[ChatAgentMessage(role='assistant', content='Yes, we have talked about Databricks before. Based on our previous conversations, we\'ve discussed:\n\n1. You asked about "Databricks Genie" in a previous conversation\n2. You inquired about "Databricks Photon" \n\nFor the Photon query, I provided you with detailed information about Databricks Photon, explaining that it\'s a next-generation query engine developed by Databricks that accelerates SQL and DataFrame workloads on the Databricks Lakehouse Platform. I shared its key features including:\n- It\'s a C++ vectorized query engine compatible with Apache Spark APIs\n- Delivers up to 12x faster SQL query performance compared to standard Spark SQL\n- Requires no code changes to implement\n- Is optimized for cloud infrastructure and modern CPU architectures\n\nIs there something specific about Databricks you\'d like to discuss further?', name='General', id='0c99a75f-2465-431d-b3d3-a6f039f71b92', tool_calls=None, tool_call_id=None, att

Trace(trace_id=tr-db9d377e0b35746cca62a9d0581c7fa5)

In [0]:
input_example_no_filter = {"messages": [ {"role": "user", "content": "What is my average active opportunity size?"}]}
answer_no_filter = AGENT.predict(input_example_no_filter)
print(answer_no_filter)

messages=[ChatAgentMessage(role='assistant', content='|    |   avg_opportunity_amount |\n|---:|-------------------------:|\n|  0 |                  85339.4 |', name='Genie', id='1a20b2ed-5afc-479f-bc61-a1212a1cd24c', tool_calls=None, tool_call_id=None, attachments=None), ChatAgentMessage(role='assistant', content='Based on the data provided, your average active opportunity size is $85,339.40.', name=None, id='run--cd5923c3-7bc8-40dc-be8a-43943bdfdc62-0', tool_calls=None, tool_call_id=None, attachments=None)] finish_reason=None custom_outputs=None usage=None


Trace(trace_id=tr-b6664716e34c106f900ed552dd8aa89f)

In [0]:
input_example = {"messages": [ {"role": "user", "content": "What was my chat history idea?"}], "custom_inputs": {"filters": {"user_name": "tanner.wendland@databricks.com"}}}
answer = AGENT.predict(input_example)
print(answer)

messages=[ChatAgentMessage(role='assistant', content='Based on searching your chat history, I don\'t see a specific "chat history idea" that you previously mentioned. The search results show that we\'ve had conversations about:\n\n1. AdTech and various databases used in that industry\n2. A question about whether we had discussed "Databricks Genie" before\n\nIf you\'re referring to a specific idea that isn\'t showing up in these results, could you provide more details about what you\'re looking for? Perhaps mentioning keywords related to the idea would help me find the relevant conversation in your chat history.', name='General', id='f3dad83f-8003-4c55-9ed2-740147ad0e3f', tool_calls=None, tool_call_id=None, attachments=None), ChatAgentMessage(role='assistant', content='I don\'t have access to other assistant messages outside of our current conversation. Based solely on our exchange so far, I can see that you asked about your "chat history idea," but I don\'t have any information about w

Trace(trace_id=tr-05aee7a2368648e609105113080fd9dc)

## Chain PY

In [0]:
from mlflow.models.resources import DatabricksVectorSearchIndex, DatabricksServingEndpoint, DatabricksGenieSpace

chain_file_path = os.path.join(os.getcwd(), 'chain_postgres_genie.py')
if not os.path.exists(chain_file_path):
    raise FileNotFoundError(f"Chain file not found at {chain_file_path}")

workspace_url = spark.conf.get("spark.databricks.workspaceUrl")
os.environ['DATABRICKS_HOST'] = f"https://{workspace_url}"
os.environ['DATABRICKS_TOKEN'] = dbutils.entry_point.getDbutils().notebook().getContext().apiToken().get()

# Log the model to MLflow
with mlflow.start_run(run_name="adtech_chat_history_agent_postgres_genie"):
  logged_chain_info = mlflow.langchain.log_model(
          #Note: In classical ML, MLflow works by serializing the model object.  In generative AI, chains often include Python packages that do not serialize.  Here, we use MLflow's new code-based logging, where we saved our chain under the chain notebook and will use this code instead of trying to serialize the object.
          lc_model=os.path.join(os.getcwd(), 'chain_postgres_genie.py'),  # Chain code file e.g., /path/to/the/chain.py 
          model_config=chain_config, # Chain configuration 
          artifact_path="chain_postgres_genie", # Required by MLflow, the chain's code/config are saved in this directory
          input_example=input_example,
          # Specify resources for automatic authentication passthrough
          resources=[
            DatabricksServingEndpoint(endpoint_name=model_config.get("llm_model_serving_endpoint_name")),
            DatabricksGenieSpace(genie_space_id=model_config.get("genie_space_id"))
          ],
          pip_requirements=[
            "mlflow==3.1.0",
            "databricks-agents==1.2.0",
            "databricks-langchain==0.7.0",
            "langchain==0.3.27",
            "pgvector==0.2.5",
            "psycopg2-binary==2.9.7",
            "pydantic==2.11.7",
            "sqlalchemy==2.0.43",
            "tornado==6.3.2",
            "langgraph==0.3.4"
          ]
      )

model_name = "chat_history_agent_postgres_genie"
MODEL_NAME_FQN = f"{target_catalog}.{target_schema}.{model_name}"
# Register to UC
uc_registered_model_info = mlflow.register_model(model_uri=logged_chain_info.model_uri, name=MODEL_NAME_FQN)

2025/08/20 21:32:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
🔗 View Logged Model at: https://e2-demo-field-eng.cloud.databricks.com/ml/experiments/617970317606308/models/m-2eab50a445574dd9a45635fc391ed021?o=1444828305810485
2025/08/20 21:33:03 WARNING mlflow.models.signature: Failed to infer model output schema from prediction result, setting output schema to AnyType. For full traceback, set logging level to debug.
2025/08/20 21:33:05 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-2eab50a445574dd9a45635fc391ed021
2025/08/20 21:33:05 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/08/20 21:33:18 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `M

Uploading artifacts:   0%|          | 0/14 [00:00<?, ?it/s]

🔗 Created version '1' of model 'tanner_wendland.default.chat_history_agent_postgres_genie': https://e2-demo-field-eng.cloud.databricks.com/explore/data/models/tanner_wendland/default/chat_history_agent_postgres_genie/version/1?o=1444828305810485


## Deploy

In [0]:
dbutils.widgets.text("secert_scope", "field-eng", "Secret Scope")
dbutils.widgets.text("secret_key", "app-secret", "Secret Key")

In [0]:
secret_scope = dbutils.widgets.get("secert_scope")
secret_key = dbutils.widgets.get("secret_key")

secret_value = dbutils.secrets.get(scope=secret_scope, key=secret_key)

print(str(secret_value))

[REDACTED]


In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import EndpointCoreConfigInput, ServedEntityInput

workspace_client = WorkspaceClient()

version = uc_registered_model_info.version
serving_endpoint_name = MODEL_NAME_FQN.replace(".", "-")

workspace_url = spark.conf.get("spark.databricks.workspaceUrl")

config = {
        "served_entities": [
            {
                "name": serving_endpoint_name,
                "entity_name": MODEL_NAME_FQN,
                "entity_version": version,
                "workload_size": "Small",
                "scale_to_zero_enabled": True,
                "environment_vars": {
                    'DATABRICKS_HOST': workspace_url,
                    'DATABRICKS_TOKEN': secret_value
                }
            }
        ]
    }

def does_endpoint_exists(endpoint_name):
    try:
        workspace_client.serving_endpoints.get(endpoint_name)
        return True
    except:
        return False

if not does_endpoint_exists(serving_endpoint_name):
    print(f"Creating endpoint {serving_endpoint_name}...")
    workspace_client.serving_endpoints.create_and_wait(
        serving_endpoint_name,
        config=EndpointCoreConfigInput.from_dict(config)
    )
else:
    print(f"Updating endpoint {serving_endpoint_name}...")
    workspace_client.serving_endpoints.update_config_and_wait(
        serving_endpoint_name,
        served_entities=[ServedEntityInput.from_dict(entity) for entity in config['served_entities']]
    )

Creating endpoint tanner_wendland-default-chat_history_agent_postgres_genie...
